In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")




In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001751E97E310>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001751EAC7010>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
# humanmsg indicates user is giving msg

model.invoke([HumanMessage(content="I am ishank, i like playing and building projects")])

AIMessage(content="That's awesome, Ishank!  \n\nWhat kind of projects do you like to build? \n\nDo you like working with your hands, using tools, or are you more into coding or designing? \n\nTell me more about your favorite projects!  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 20, 'total_tokens': 77, 'completion_time': 0.103636364, 'prompt_time': 0.00021314, 'queue_time': 0.025576619, 'total_time': 0.103849504}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c08d0563-68da-44e2-8ba1-a6d252e69d5d-0', usage_metadata={'input_tokens': 20, 'output_tokens': 57, 'total_tokens': 77})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="I am ishank, i like playing and building projects"),
        AIMessage(content="That's awesome, Ishank!  \n\nWhat kind of projects do you like to build? \n\nDo you like working with your hands, using tools, or are you more into coding or designing? \n\nTell me more about your favorite projects!"),
        HumanMessage(content="what is my name and what do i like to do?")
    ]
)

AIMessage(content="You are Ishank, and you like to play and build projects! \n\nIs there anything specific you'd like to talk about regarding your projects or hobbies? I'm happy to chat more! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 94, 'total_tokens': 140, 'completion_time': 0.083636364, 'prompt_time': 0.002496342, 'queue_time': 0.011286998, 'total_time': 0.086132706}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e9ff819d-d623-480e-ac64-17f56bbd2f49-0', usage_metadata={'input_tokens': 94, 'output_tokens': 46, 'total_tokens': 140})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# runnables for chain



1. ChatMessageHistory : Manages and stores chat history within LangChain for tracking conversation state across interactions.
2. RunnableWithMessageHistory : Executes a runnable (e.g., a chain or function) while keeping track of the associated chat message history for further processing or evaluation.
3. BaseChatMessageHistory : if you wanted to store chat history in a database, you could create a class that inherits from BaseChatMessageHistory and implements the required methods for saving and retrieving chat messages.

In [92]:
store={}
# fn to retrieve chagt history based on session id
def get_session_history(session_id:str)->BaseChatMessageHistory:
# ses id used to distinuish sessions
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]   

with_message_history=RunnableWithMessageHistory(model,get_session_history)


If the session ID is not in the store:

It creates a new ChatMessageHistory object for that session.
This new history is stored in the store dictionary under the key session_id.
Returns the session's chat history:

If the session already exists, it returns the previously stored history.
If it's a new session, it returns the newly created empty chat history.

In [93]:
config={"configurable":{"session_id":"chat1"}}
# These key names are part of the internal structure that LangChain uses to process configurations and manage sessions.

In [21]:
with_message_history.invoke(
    [HumanMessage(content='I am ishank, i like playing and building projects')],
    config=config
)

AIMessage(content="It's great to meet you, Ishank!  \n\nI'm curious, what kind of projects do you like to build? 🛠️\n\nDo you enjoy working with wood, electronics, coding, or something totally different? \n\nTell me about a project you're really proud of!  😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 92, 'total_tokens': 161, 'completion_time': 0.125454545, 'prompt_time': 0.006746988, 'queue_time': 0.006379762, 'total_time': 0.132201533}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6f20f320-958f-4bd8-a4fb-fc84a29d8c20-0', usage_metadata={'input_tokens': 92, 'output_tokens': 69, 'total_tokens': 161})

In [23]:
with_message_history.invoke(
    [HumanMessage(content='what is my name?')],
    config=config
)

AIMessage(content="Your name is Ishank! 😊  \n\nIs there anything else you'd like to talk about, Ishank?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 256, 'total_tokens': 283, 'completion_time': 0.049090909, 'prompt_time': 0.037690938, 'queue_time': 0.050889291999999996, 'total_time': 0.086781847}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7f49ce5b-5f77-4095-9e31-966470df1c15-0', usage_metadata={'input_tokens': 256, 'output_tokens': 27, 'total_tokens': 283})

In [29]:
# Changing the config name, chat is kept same
config22={"configurable":{"session_id":"chat1"}}

response=with_message_history.invoke(
    [HumanMessage(content='what is my name?')],
    config=config22
)
response.content

'Your name is Ishank.  \n\nDid you have something else in mind? 😊 \n'

In [97]:
# Changing the config session id
# chat2
config22={"configurable":{"session_id":"chat2"}}

response=with_message_history.invoke(
    [HumanMessage(content='what is my name?')],
    config=config22
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to know! 😊  \n\n"

In [98]:
response=with_message_history.invoke(
    [HumanMessage(content='My name is John')],
    config=config22
)
response.content

"Hi John, it's nice to meet you! 👋 \n\nIs there anything I can help you with today?\n"

In [99]:
response=with_message_history.invoke(
    [HumanMessage(content='what is my name ?')],
    config=config
    # config=config22
)
response.content

"Your name is Ishank.  \n\nI'm happy to help remind you! 😊  \n\nIs there something else you'd like to talk about?  \n\n\n"

#### When you change the session_id: You are either:
Continuing a chat session by referencing its past messages.
<!--  -->
Starting a new chat session by providing a new session_id.

## Prompt Template

In [41]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
        # MessagesPlaceholder to dynamically insert the chat history
    ]
)
# key val pairs 
chain =prompt|model

In [43]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Mike")]})

AIMessage(content="Hello Mike! It's nice to meet you.  \n\nHow can I help you today? 😄 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 31, 'total_tokens': 56, 'completion_time': 0.045454545, 'prompt_time': 0.000341108, 'queue_time': 0.012812282, 'total_time': 0.045795653}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-335d887f-5927-46c7-8cda-cfef14ff45eb-0', usage_metadata={'input_tokens': 31, 'output_tokens': 25, 'total_tokens': 56})

In [44]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)
# chain is prompt and model

# RunnableWithMessageHistory: This is a class from LangChain that adds message history functionality to a runnable object (in this case, chain).

# chain: This is the actual runnable chain that processes inputs (messages) and generates responses.
# get_session_history: This function is used to retrieve the session history for a specific session_id.

In [45]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi my name is Mike")],
    config=config,
    )
response

AIMessage(content="Hello Mike! It's nice to meet you.\n\nWhat can I do for you today?  😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 31, 'total_tokens': 57, 'completion_time': 0.047272727, 'prompt_time': 0.000336829, 'queue_time': 0.012969172, 'total_time': 0.047609556}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-47daa44e-97bf-497b-81bd-b8cb2b40b86b-0', usage_metadata={'input_tokens': 31, 'output_tokens': 26, 'total_tokens': 57})

In [46]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [47]:
response=chain.invoke({"messages":[HumanMessage(content="I am superman")],"language":"Hindi"})
response.content

'नमस्ते सुपरमैन! 🦸\u200d♂️\n\nमैं आपकी मदद करने के लिए यहाँ हूँ। आप मुझसे कोई भी प्रश्न पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा कि आपको सबसे अच्छी जवाब दे सकूँ। 😊 \n\nआप क्या जानना चाहते हैं? \n'

In [48]:
# now wrapping this chain in message history class

with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"

)

In [56]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="My name is batman")],"language":"Hindi"},
    config=config
)
response.content


'हाँ, आपका नाम Batman है!  😊  \n\nआपको कुछ और चाहिए?  \n'

In [78]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="What is my name?")],"language":"Hindi"},
    config=config
)
response.content


'आपका नाम Mike है।  \n'

In [55]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="What is my name?")],"language":"Hindi"},
    config=config
)
response.content


'आपका नाम Batman है।  \n\nक्या आप कोई और सवाल पूछना चाहते हैं? \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
<!--  -->
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [58]:
from langchain_core.messages import SystemMessage,trim_messages
# trim- .. helper helps reduce how many messages we are sending to model
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
# focus on last convo it had , count from there
token_counter=model,
include_system=True,
allow_partial=False,
start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

# top 2 messages trimmed

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

d:\GenAi\Langchain\langvenv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ishan\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

d:\GenAi\Langchain\langvenv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [65]:
itemgetter("messages")|trimmer

RunnableLambda(itemgetter('messages'))
| RunnableLambda(...)

### Making a Chain with trimmer function


In [81]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

# n LangChain, RunnablePassthrough is a type of Runnable that essentially passes its input directly to its output without modifying or transforming it

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    | model

)
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What math problem did i ask?" )],
    "language":"English"
    }

)

response.content

'You asked "What is 2 + 2?"  😊 \n\n\nLet me know if you want to try another one!\n'

In [82]:
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config={"configurable":{"session_id":"chat5"}}